<a href="https://colab.research.google.com/github/stellagerantoni/LatentCfMultivariate/blob/main/FaceDetection_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/stellagerantoni/LatentCfMultivariate

Cloning into 'LatentCfMultivariate'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 212 (delta 80), reused 36 (delta 36), pack-reused 105
Receiving objects: 100% (212/212), 12.40 MiB | 23.64 MiB/s, done.
Resolving deltas: 100% (120/120), done.


In [2]:
!pip install -q wildboar
!pip install -q scikit-learn
!pip install -q stumpy
!pip install -q fastdtw
!pip install aeon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 926.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 17.0 MB/s eta 0:00:00


In [3]:
import logging
import os
import warnings
from argparse import ArgumentParser
from aeon.datasets import load_classification

from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.spatial import distance_matrix
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree, KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from wildboar.datasets import load_dataset
from wildboar.ensemble import ShapeletForestClassifier
from wildboar.explain.counterfactual import counterfactuals
%cd '/content/LatentCfMultivariate'
from _guided import ModifiedLatentCF
from help_functions import *
from keras_models import *

/content/LatentCfMultivariate


In [4]:
os.environ['TF_DETERMINISTIC_OPS'] = '1'
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
RANDOM_STATE = 39

# **ACTUALL CODE**
datasets available : 'Heartbeat', 'SelfRegulationSCP1'

In [37]:
def load_dataset(dataset):
  X, y = load_classification(dataset)
  if dataset == 'FaceDetection':
    pos = '1'
    neg = '0'


  print(" Shape of X = ", X.shape)
  print(" Shape of y = ", y.shape)
  #print(" Meta data = ", meta_data)
  # Convert positive and negative labels to 1 and 0
  pos_label, neg_label = 1, 0
  if pos != pos_label:
      y[y==pos] = pos_label # convert/normalize positive label to 1
  if neg != neg_label:
      y[y==neg] = neg_label # convert negative label to 0

  y = y.astype(int)
  print(f"\n X[:1] = \n{X[:1]}")
  return X,y


In [38]:
RANDOM_STATE = 39
X, y = load_dataset('FaceDetection')
X = X.transpose(0,2,1)

#print(f'data imformation = {data_information}')
#keep half the dataset because it is too big
X,X_through,y,y_through = train_test_split(X, y, test_size=0.5, random_state=RANDOM_STATE, stratify=y)
print(f'shape of X = {X.shape}')
print(f'shape of y = {y.shape}')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
print(f'shape of X train = {X_train.shape}')
print(f'shape of y train = {y_train.shape}')

 Shape of X =  (9414, 144, 62)
 Shape of y =  (9414,)

 X[:1] = 
[[[-0.07545  -0.336703 -0.278238 ... -0.411078 -1.016122 -1.161735]
  [ 0.05608  -0.128013 -0.323847 ... -2.114348  0.208789 -0.509533]
  [-0.824537 -0.746068 -0.482871 ... -0.929275 -1.007972 -0.292018]
  ...
  [-0.56758  -1.073942 -1.136367 ... -0.541122 -0.765445 -1.73308 ]
  [-0.23404   0.104291  0.327425 ... -1.458801  0.318952  1.854007]
  [-0.356189 -0.511199 -0.483072 ... -1.177502 -0.728301 -0.400074]]]
shape of X = (4707, 62, 144)
shape of y = (4707,)
shape of X train = (3765, 62, 144)
shape of y train = (3765,)


In [39]:
#Upsample the minority class

unique_classes, class_counts = np.unique(y_train, return_counts=True)
print(f'before: {class_counts}')
X_train,y_train = upsample_minority_multivariate(X_train,y_train)
X,y = upsample_minority_multivariate(X, y)
unique_classes, class_counts = np.unique(y_train, return_counts=True)
print(f'after: {class_counts}')

before: [1883 1882]
after: [1883 1883]


In [40]:
#Processing and Padding all our data
#Padding needed for autoencoder

n_training,n_timesteps, n_features= X_train.shape

X, trained_scaler =  normalize_multivariate(data=X, n_timesteps=n_timesteps, n_features = n_features)
X_train_processed, trained_scaler =  normalize_multivariate(data=X_train, n_timesteps=n_timesteps, n_features = n_features)
X_test_processed, _ =  normalize_multivariate(data=X_test, n_timesteps=n_timesteps, scaler=trained_scaler, n_features = n_features)

X, padding_size = conditional_pad_multivariate(X)
X_train_processed_padded, padding_size = conditional_pad_multivariate(X_train_processed) # add extra padding zeros if n_timesteps cannot be divided by 4, required for 1dCNN autoencoder structure
X_test_processed_padded, _ = conditional_pad_multivariate(X_test_processed)

n_timesteps_padded = X_train_processed_padded.shape[1]
print(f"Data pre-processed, original #timesteps={n_timesteps}, padded #timesteps={n_timesteps_padded}.")

#check the processing (0,1) min should be min 0 and max should be max 1
print(f"\nmin value = {np.min(X_train)}, max value = {np.max(X_train)}")
print(f"min value normalized = {np.min(X_train_processed)}, max value normalized= {np.max(X_train_processed)}")

#check that padding paddes the right dimention
print(f"\nX_train.shape = {X_train.shape}" )
print(f"X_train_processed_padded.shape = {X_train_processed_padded.shape}")


Data pre-processed, original #timesteps=62, padded #timesteps=64.

min value = -24.327769, max value = 24.326942
min value normalized = 0.0, max value normalized= 1.0000000000000002

X_train.shape = (3766, 62, 144)
X_train_processed_padded.shape = (3766, 64, 144)


In [41]:
#splitting the dataset

from sklearn.model_selection import train_test_split
X_train,X_validation, y_train, y_validation = train_test_split(X_train_processed_padded, y_train, test_size=0.2, random_state=RANDOM_STATE, stratify=y_train)

In [42]:
#Getting the two forms of labels needed
#-the y_classes (1,0,1,0,...)
#-the y (one hot encoded)

print(f'X_train = {X_train.shape}')
print(f'X_validation = {X_validation.shape}')
print(f'X_test = {X_test.shape}')

y_classes = y
y_train_classes = y_train
y_validation_classes = y_validation
y_test_classes = y_test

from tensorflow.keras.utils import to_categorical
y = to_categorical(y, len(np.unique(y)))
y_train = to_categorical(y_train, len(np.unique(y_train)))
y_validation = to_categorical(y_validation, len(np.unique(y_validation)))
y_test = to_categorical(y_test, len(np.unique(y_test)))

print(f'\ny_train_classes = {y_train_classes.shape}, y_validation_classes = {y_validation_classes.shape}, y_test_classes = {y_test_classes.shape}')
print(f'y_train = {y_train.shape}, y_validation = {y_validation.shape}, y_test= {y_test.shape}')

X_train = (3012, 64, 144)
X_validation = (754, 64, 144)
X_test = (942, 62, 144)

y_train_classes = (3012,), y_validation_classes = (754,), y_test_classes = (942,)
y_train = (3012, 2), y_validation = (754, 2), y_test= (942, 2)


In [11]:
def Classifier(
    n_timesteps, n_features, n_conv_layers=1, add_dense_layer=True, n_output=2
):
    # https://keras.io/examples/timeseries/timeseries_classification_from_scratch/

    input_shape = ( n_timesteps,n_features)

    inputs = keras.Input(shape=input_shape, dtype="float32")

    if add_dense_layer:
        x = keras.layers.Dense(128, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(inputs)
    else:
        x = inputs

    for i in range(n_conv_layers):
        x = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.ReLU()(x)

    x = keras.layers.MaxPooling1D(pool_size=2, padding="same")(x)
    x = keras.layers.Flatten()(x)

    if n_output >= 2:
        outputs = keras.layers.Dense(n_output,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation="softmax")(x)
    else:
        outputs = keras.layers.Dense(1, activation="sigmoid")(x)

    classifier = keras.models.Model(inputs=inputs, outputs=outputs)

    return classifier

In [12]:


# ## LatentCF++ models
# reset seeds for numpy, tensorflow, python random package and python environment seed
reset_seeds()
###############################################
# ### 1dCNN classifier

cnnClassifier = Classifier(
    n_timesteps_padded, n_features, n_output=2, add_dense_layer = False
)

optimizer = keras.optimizers.Adam(lr=0.001)
cnnClassifier.compile(
    optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
)

# Define the early stopping criteria
early_stopping_accuracy = keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=15, restore_best_weights=True
)
# Train the model
reset_seeds()
print("Training log for LSTM-FCN classifier:")
classifier_history = cnnClassifier.fit(
    X_train,
    y_train,
    epochs=150,
    batch_size=12,
    shuffle=True,
    verbose=True,
    validation_data=(X_validation, y_validation),
    callbacks=[early_stopping_accuracy],
)

y_pred = cnnClassifier.predict(X_test_processed_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
acc = balanced_accuracy_score(y_true=y_test_classes, y_pred=y_pred_classes)
print(f"LSTM-FCN classifier trained, with validation accuracy {acc}.")

confusion_matrix_df = pd.DataFrame(
    confusion_matrix(y_true=y_test_classes, y_pred=y_pred_classes, labels=[1, 0]),
    index=["True:1", "True:0"],
    columns=["Pred:1", "Pred:0"],
)
print(confusion_matrix_df)


Training log for LSTM-FCN classifier:
Epoch 1/150
251/251 [==============================] - 7s 7ms/step - loss: 0.7165 - accuracy: 0.5697 - val_loss: 0.8381 - val_accuracy: 0.5000
Epoch 2/150
251/251 [==============================] - 1s 5ms/step - loss: 0.5930 - accuracy: 0.7178 - val_loss: 0.7679 - val_accuracy: 0.5027
Epoch 3/150
251/251 [==============================] - 1s 6ms/step - loss: 0.4858 - accuracy: 0.7985 - val_loss: 0.6333 - val_accuracy: 0.6724
Epoch 4/150
251/251 [==============================] - 1s 5ms/step - loss: 0.4049 - accuracy: 0.8612 - val_loss: 1.3723 - val_accuracy: 0.5040
Epoch 5/150
251/251 [==============================] - 1s 5ms/step - loss: 0.3287 - accuracy: 0.9017 - val_loss: 1.0553 - val_accuracy: 0.5345
Epoch 6/150
251/251 [==============================] - 1s 5ms/step - loss: 0.2693 - accuracy: 0.9376 - val_loss: 1.7538 - val_accuracy: 0.5080
Epoch 7/150
251/251 [==============================] - 2s 6ms/step - loss: 0.2304 - accuracy: 0.9552 - v

In [13]:
reset_seeds()

# ### 1dCNN autoencoder
autoencoder = Autoencoder( n_timesteps_padded,n_features,32)
optimizer = keras.optimizers.Adam(lr=0.0005)
autoencoder.compile(optimizer=optimizer, loss="mse")

# Define the early stopping criteria
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, restore_best_weights=True)
# Train the model
reset_seeds()
print("Training log for 1dCNN autoencoder:")
autoencoder_history = autoencoder.fit(
    X_train,
    X_train,
    epochs=50,
    batch_size=12,
    shuffle=True,
    verbose=2,
    validation_data=(X_validation, X_validation),
    callbacks=[early_stopping])

ae_val_loss = np.min(autoencoder_history.history['val_loss'])
print(f"1dCNN autoencoder trained, with validation loss: {ae_val_loss}.")


(None, 64, 144)
(None, 64, 32)
(None, 32, 32)
(None, 32, 16)
(None, 16, 16)
(None, 16, 16)
(None, 32, 16)


(None, 32, 32)
(None, 64, 32)
(None, 64, 144)
Training log for 1dCNN autoencoder:
Epoch 1/50
251/251 - 5s - loss: 0.0144 - val_loss: 0.0016 - 5s/epoch - 18ms/step
Epoch 2/50
251/251 - 2s - loss: 0.0012 - val_loss: 0.0012 - 2s/epoch - 9ms/step
Epoch 3/50
251/251 - 2s - loss: 0.0011 - val_loss: 0.0012 - 2s/epoch - 8ms/step
Epoch 4/50
251/251 - 2s - loss: 0.0010 - val_loss: 0.0011 - 2s/epoch - 7ms/step
Epoch 5/50
251/251 - 2s - loss: 0.0010 - val_loss: 0.0011 - 2s/epoch - 7ms/step
Epoch 6/50
251/251 - 2s - loss: 0.0010 - val_loss: 0.0011 - 2s/epoch - 7ms/step
Epoch 7/50
251/251 - 2s - loss: 0.0010 - val_loss: 0.0011 - 2s/epoch - 7ms/step
1dCNN autoencoder trained, with validation loss: 0.001126678311266005.


In [14]:
#Gettting the Global weights, needed for counterfactuals

from _guided import get_global_weights
from help_functions import evaluate
pos_label = 1
neg_label = 0

step_weights = get_global_weights(
        X,
        y_classes,
        cnnClassifier,
        n_timesteps= n_timesteps,
        n_features=n_features,
        random_state=RANDOM_STATE,
)


148/148 [==============================] - 1s 3ms/step


In [15]:
step_weights

array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]])

In [43]:
reset_seeds()
cf_model = ModifiedLatentCF(
    probability=0.5,tolerance=1e-6, max_iter=500, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),autoencoder = autoencoder,
    pred_margin_weight=0.7, step_weights = step_weights, random_state= RANDOM_STATE)
cf_model.fit(cnnClassifier)

y_neg = y_classes[y_classes == 0][10:15]
X_neg = X[y_classes == 0][10:15]


with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning) # ignore warning of matrix multiplication: https://stackoverflow.com/questions/29688168/mean-nanmean-and-warning-mean-of-empty-slice
    cf_embeddings, losses, weights = cf_model.transform(X_neg, y_neg) #self, x, pred_label
cf_pred_labels = cnnClassifier.predict(cf_embeddings)[:,1]# predicted probabilities of CFs
for idx in range(cf_pred_labels.shape[0]):
  if cf_pred_labels[idx] > 0.5:
    cf_pred_labels[idx] = 1
  else:
    cf_pred_labels[idx] = 0

print(f'Transformation_finished with validity_score = {validity_score(y_neg,cf_pred_labels)}')

1 samples been transformed.
5 samples been transformed, in total.
1/1 [==============================] - 0s 76ms/step
Transformation_finished with validity_score = 1.0


In [44]:
#Calculating proximity
from tensorflow.keras.losses import MeanSquaredError
total = 0
probability = 0.5
for idx in range(cf_embeddings.shape[0]):
    counterfactual = cf_embeddings[idx,np.newaxis]
    prediction = cnnClassifier.predict(counterfactual)[:, 1]
    dist = (prediction - probability)
    total +=dist
mean_mse = total /cf_embeddings.shape[0]


1/1 [==============================] - 0s 30ms/step


In [45]:
print(f"The Mean MSE of the data is: {mean_mse} ")

The Mean MSE of the data is: [0.00062653] 


In [46]:
#Calculating proximity
from tensorflow.keras.losses import MeanSquaredError
total = 0
probability = 0.5
for idx in range(cf_embeddings.shape[0]):
    counterfactual = cf_embeddings[idx,np.newaxis]
    prediction = cnnClassifier.predict(counterfactual)[:, 1]
    dist = abs(prediction - probability)
    total +=dist
mean_mse = total /cf_embeddings.shape[0]


1/1 [==============================] - 0s 41ms/step


In [47]:
print(f"The Absolute Mean MSE of the data is: {mean_mse} ")

The Absolute Mean MSE of the data is: [0.00062653] 


In [48]:
#Proximity
def euclidean_distance(X, cf_samples):
    paired_distances = np.linalg.norm(X - cf_samples, axis=1)
    return np.mean(paired_distances)
euclidean_distance(X_neg, cf_embeddings)

0.6309590391658204

In [49]:
def remove_paddings(cf_samples, padding_size):
    if padding_size != 0:
        # use np.squeeze() to cut the last time-series dimension, for evaluation
        cf_samples = np.squeeze(cf_samples[:, :-padding_size, :])
    else:
        cf_samples = np.squeeze(cf_samples)
    return cf_samples

In [50]:
# Remove paddings because KDE does not work with paddings.

X_unpadded = remove_paddings(X, padding_size)
cf_embeddings_unpadded = remove_paddings(cf_embeddings, padding_size)

In [53]:
from scipy.stats import gaussian_kde
diffrences_from_abnormal = []
diffrences_from_normal = []
for dimention in range(cf_embeddings.shape[2]):


  abnormal_data = X_unpadded[y_classes == 1][:,:,dimention]
  normal_data = X_unpadded[y_classes == 0][:,:,dimention]
  counterf_data = cf_embeddings_unpadded[:,:,dimention]

  #get the kernel for every dimention of the trained
  kernel = gaussian_kde(abnormal_data.T,bw_method=None)

  #get all the log likelihoods
  log_likelihood_abnormal = np.mean(kernel.logpdf(abnormal_data.T))
  log_likelihood_normal = np.mean(kernel.logpdf(normal_data.T))
  log_likelihood_counterfactual = np.mean(kernel.logpdf(counterf_data.T))

  #get the diffrences from the counterfactuals
  diff_from_abnormal = abs(log_likelihood_counterfactual-log_likelihood_abnormal)
  diffrences_from_abnormal.append(diff_from_abnormal)

  diff_from_normal = abs(log_likelihood_counterfactual-log_likelihood_normal)
  diffrences_from_normal.append(diff_from_normal)



In [54]:
print(diffrences_from_normal)

[892.7991458090487, 507.48372717440674, 7212.293084950014, 3958.5777792982944, 2099.423277455748, 1611.05941927837, 165.18937917500892, 1822.2082178846972, 9589.997777367811, 152.54085558694976, 4013.7445440100078, 4718.30681153119, 633.267428552139, 503.42296635799187, 600.3581203729711, 503.651819148032, 355.0375695626642, 6437.292804080113, 1423.5699891917907, 5479.180789471258, 3398.933327073883, 2806.7413071480673, 5075.291598788244, 6253.685246278098, 802.6168025919274, 1783.8260108565705, 16.85973596370178, 531.4545656211949, 12818.045648106998, 2154.1262160324636, 382.64368370819966, 963.1352973523128, 378.94386723402806, 197.26409986287592, 1897.9236473696462, 633.5152151772783, 380.8124957713888, 958.7483504668301, 2668.741340940481, 647.8920008942112, 5174.720411037899, 2187.1670672468144, 5318.649274763037, 460.3466538873888, 1722.1228666513516, 467.49061963868866, 565.9474088245365, 12840.638502090847, 3336.3421812183788, 3581.144993835633, 4285.412114483446, 226.909518354

In [55]:
print(diffrences_from_abnormal)

[933.7437609518607, 550.0356467077538, 7255.690818955082, 3999.486812125629, 2138.9283884713145, 1654.8338847057285, 205.69768172354614, 1862.2773736106774, 9628.994373357355, 193.6650942326363, 4057.649927798447, 4758.0334041692695, 672.7045962495133, 543.3037580202272, 641.6607400740069, 539.8355213239021, 390.7068996600273, 6474.526901237176, 1464.2072304868768, 5517.516816821971, 3438.638718853822, 2848.29733521771, 5114.522623438901, 6294.656412630652, 842.9367151990947, 1822.137055622844, 56.223781822496335, 573.0719927973307, 12855.553071152304, 2194.7081023108376, 423.33744477418344, 1000.9707326848865, 415.685998760493, 237.30610143694793, 1936.3254118649068, 673.1364669225461, 417.7100147474101, 995.1943658933012, 2704.297305903219, 684.8462217809255, 5212.762508733597, 2227.442028815562, 5355.485777942547, 496.0628611529573, 1758.9217132253568, 508.17285213165883, 604.381199130286, 12880.752516447466, 3377.993924942566, 3617.8386152052894, 4324.343854746299, 267.539822297539

In [56]:
print(np.mean(diffrences_from_normal))

1855.2384038996638


In [57]:
print(np.mean(diffrences_from_abnormal))

1894.717835466105
